## libraries and function 


In [1]:
# !pip install impyute
!pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time
# !pip install missingpy
# from missingpy import MissForest
# note that MissForest uses sklearn.__version__ 0.22.2.post1

     |████████████████████████████████| 23.1 MB 1.9 MB/s 
     |████████████████████████████████| 2.7 MB 37.1 MB/s 
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11353 sha256=0daa0beb77561dcc1b3e691f0fbc74fb93083defb175a8dd6c30b3c7720464db
  Stored in directory: /root/.cache/pip/wheels/72/21/a8/a045cacd9838abd5643f6bfa852c0796a99d6b1494760494e0
Successfully built knnimpute
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


## MLE estimation function  




In [2]:
def diag_term(i,X,y):
  arr0 = X[:,i]
  nar2 = 0
  arr = arr0[~np.isnan(arr0)]
  y_arr = y[~np.isnan(arr0)]

  _, counts = np.unique(y_arr, return_counts=True)
  ind = np.insert(np.cumsum(counts), 0, 0)
  
  return sum([(ind[g]-ind[g-1])*np.var(arr[ind[g-1]:ind[g]]) for 
                       g in range(1,G+1)])/len(y_arr)                       

In [3]:
def mle(X,y,G):
    '''
    X: input, should be a numpy array
    y: label
    G: number of classes
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    n,p = X.shape[0], X.shape[1]

    # Estimating class means
    mus = np.array([np.nanmean(X[y==g,:],axis=0) for g in range (G)]).T # so that each column is the mean of a class
 
    S = np.diag([diag_term(i,X,y) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        mat = X[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], y[idx]

        _, counts = np.unique(y_arr, return_counts=True)
        ind = np.insert(np.cumsum(counts), 0, 0)

        m_g = counts
 
        A = len(y_arr) 
        scaled_mat = [mat[ind[g-1]:ind[g],:]-mus[[i,j],g-1] for g in range(1,G+1)]   

        q = lambda g: np.dot(scaled_mat[g][:,0],scaled_mat[g][:,0])
        s11 = sum(map(q,range(G))) 
        q = lambda g: np.dot(scaled_mat[g][:,1],scaled_mat[g][:,1])
        s22 = sum(map(q,range(G))) 
        d = lambda g: np.dot(scaled_mat[g][:,0],scaled_mat[g][:,1])
        s12 = sum(map(d,range(G)))  

        start_solve = time.time()
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)

        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0

        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 +
                                      r**2/S[i,i]**2*s11)/condi_var
          # if condi_var <0 then eta = NA. in practice, it's impossible for cov to be negative
          #  therefore, we drop NA elements of eta  
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1:        
            w = [m_g[g-1]*np.cov(mat[ind[g-1]:ind[g],], rowvar=False) for
                 g in range(1,G+1)]
            w = np.sum(w, axis = 0)    
            r = r[np.abs(r-w[0,1]).argmin()] # select r that is closet to w[0,1] 
              
        S[i,j] = S[j,i] = r
    return [mus, S]

### compute_err function 
  

In [4]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
         np.linalg.norm(S_est.flatten()-S.flatten())/S.size]
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape)

In [5]:
def compute_err_mle(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  running_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    # MLEs approach
    start = time.time()
    mus_mle, S_mle = mle(Xtr_nan,ytrain, G)
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start    
    e_rate.append(mle_err)
    running_time.append(mle_time)
  e_rate = np.asarray(e_rate)
  running_time = np.asarray(running_time)
  return np.mean(e_rate), np.std(e_rate), np.mean(running_time)    

In [6]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  running_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([(sum(ytrain==g))*np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_softimpute = np.sum(S_softimpute, axis = 0)/len(ytrain) 
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start
    running_time.append(softimpute_time)

    e_rate.append(softimpute_err)
  e_rate = np.asarray(e_rate)
  running_time = np.asarray(running_time)
  return np.mean(e_rate), np.std(e_rate), np.mean(running_time)     

In [7]:
def compute_err_mice(Xtrain, ytrain, G, missing_rate, runs = 10):
  e_rate = []
  running_time = []
  for i in  range(runs):
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    mus = [np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)]
    mus = np.asarray(mus) # each row is a mean of a class
    S = [sum(ytrain==g)*np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]
    S = np.sum(S, axis = 0)/len(ytrain)


    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=100).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([(sum(ytrain==g))*np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_mice = np.sum(S_mice, axis = 0)/len(ytrain) 
    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start
    running_time.append(mice_time)
    e_rate.append(mice_err)
  e_rate = np.asarray(e_rate)
  running_time = np.asarray(running_time)
  return np.mean(e_rate), np.std(e_rate), np.mean(running_time)     

# Digits

In [8]:
digits = datasets.load_digits()
X,y = digits.data, digits.target.ravel() 
print(X.shape)
rmid = np.where(sum(X!=0)<10)
X = np.delete(X, rmid,axis = 1)
X.shape

(1797, 64)


(1797, 54)

In [19]:
G = 10
mle_err = np.array([compute_err_mle(X, y, G, .2, runs = 10),
                    compute_err_mle(X, y, G, .35, runs = 10),
                    compute_err_mle(X, y, G, .5, runs = 10),
                    compute_err_mle(X, y, G, .65, runs = 10),
                    compute_err_mle(X, y, G, .8, runs = 10)])
mle_err.round(3)

array([[0.002, 0.   , 0.764],
       [0.002, 0.   , 0.737],
       [0.002, 0.   , 0.716],
       [0.003, 0.   , 0.691],
       [0.003, 0.   , 0.676]])

In [9]:
G = 10
mice_err = np.array([compute_err_mice(X, y, G, .2, runs = 10),
                    compute_err_mice(X, y, G, .35, runs = 10),
                    compute_err_mice(X, y, G, .5, runs = 10),
                    compute_err_mice(X, y, G, .65, runs = 10),
                    compute_err_mice(X, y, G, .8, runs = 10)])
mice_err.round(3)

array([[1.0000e-03, 0.0000e+00, 8.2140e+00],
       [1.0000e-03, 0.0000e+00, 7.1410e+00],
       [2.0000e-03, 0.0000e+00, 4.8150e+00],
       [3.0000e-03, 0.0000e+00, 1.0678e+01],
       [4.0000e-03, 1.0000e-03, 1.1375e+01]])

In [10]:
G = 10
soft_err = np.array([compute_err_soft(X, y, G, .2, runs = 10),
                    compute_err_soft(X, y, G, .35, runs = 10),
                    compute_err_soft(X, y, G, .5, runs = 10),
                    compute_err_soft(X, y, G, .65, runs = 10),
                    compute_err_soft(X, y, G, .8, runs = 10)])
soft_err.round(3)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


[SoftImpute] Max Singular Value of X_init = 95.359334
[SoftImpute] Iter 1: observed MAE=0.035299 rank=54
[SoftImpute] Iter 2: observed MAE=0.035377 rank=54
[SoftImpute] Iter 3: observed MAE=0.035450 rank=54
[SoftImpute] Iter 4: observed MAE=0.035517 rank=54
[SoftImpute] Iter 5: observed MAE=0.035579 rank=54
[SoftImpute] Iter 6: observed MAE=0.035635 rank=54
[SoftImpute] Iter 7: observed MAE=0.035686 rank=54
[SoftImpute] Iter 8: observed MAE=0.035733 rank=54
[SoftImpute] Iter 9: observed MAE=0.035775 rank=54
[SoftImpute] Iter 10: observed MAE=0.035812 rank=54
[SoftImpute] Iter 11: observed MAE=0.035846 rank=54
[SoftImpute] Iter 12: observed MAE=0.035876 rank=54
[SoftImpute] Iter 13: observed MAE=0.035902 rank=54
[SoftImpute] Iter 14: observed MAE=0.035924 rank=54
[SoftImpute] Iter 15: observed MAE=0.035944 rank=54
[SoftImpute] Iter 16: observed MAE=0.035961 rank=54
[SoftImpute] Iter 17: observed MAE=0.035975 rank=54
[SoftImpute] Iter 18: observed MAE=0.035987 rank=54
[SoftImpute] Iter 1

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


[SoftImpute] Max Singular Value of X_init = 95.097393
[SoftImpute] Iter 1: observed MAE=0.035250 rank=54
[SoftImpute] Iter 2: observed MAE=0.035329 rank=54
[SoftImpute] Iter 3: observed MAE=0.035401 rank=54
[SoftImpute] Iter 4: observed MAE=0.035469 rank=54
[SoftImpute] Iter 5: observed MAE=0.035531 rank=54
[SoftImpute] Iter 6: observed MAE=0.035589 rank=54
[SoftImpute] Iter 7: observed MAE=0.035641 rank=54
[SoftImpute] Iter 8: observed MAE=0.035688 rank=54
[SoftImpute] Iter 9: observed MAE=0.035730 rank=54
[SoftImpute] Iter 10: observed MAE=0.035768 rank=54
[SoftImpute] Iter 11: observed MAE=0.035801 rank=54
[SoftImpute] Iter 12: observed MAE=0.035831 rank=54
[SoftImpute] Iter 13: observed MAE=0.035857 rank=54
[SoftImpute] Iter 14: observed MAE=0.035880 rank=54
[SoftImpute] Iter 15: observed MAE=0.035899 rank=54
[SoftImpute] Iter 16: observed MAE=0.035916 rank=54
[SoftImpute] Iter 17: observed MAE=0.035930 rank=54
[SoftImpute] Iter 18: observed MAE=0.035942 rank=54
[SoftImpute] Iter 1

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


...

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


[SoftImpute] Max Singular Value of X_init = 55.461412
[SoftImpute] Iter 1: observed MAE=0.028957 rank=54
[SoftImpute] Iter 2: observed MAE=0.028967 rank=54
[SoftImpute] Iter 3: observed MAE=0.028977 rank=54
[SoftImpute] Iter 4: observed MAE=0.028987 rank=54
[SoftImpute] Iter 5: observed MAE=0.028995 rank=54
[SoftImpute] Iter 6: observed MAE=0.029004 rank=54
[SoftImpute] Iter 7: observed MAE=0.029012 rank=54
[SoftImpute] Iter 8: observed MAE=0.029020 rank=54
[SoftImpute] Iter 9: observed MAE=0.029027 rank=54
[SoftImpute] Iter 10: observed MAE=0.029033 rank=54
[SoftImpute] Iter 11: observed MAE=0.029039 rank=54
[SoftImpute] Iter 12: observed MAE=0.029045 rank=54
[SoftImpute] Iter 13: observed MAE=0.029051 rank=54
[SoftImpute] Iter 14: observed MAE=0.029056 rank=54
[SoftImpute] Iter 15: observed MAE=0.029060 rank=54
[SoftImpute] Iter 16: observed MAE=0.029065 rank=54
[SoftImpute] Iter 17: observed MAE=0.029068 rank=54
[SoftImpute] Iter 18: observed MAE=0.029072 rank=54
[SoftImpute] Iter 1

/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


[SoftImpute] Max Singular Value of X_init = 55.589986
[SoftImpute] Iter 1: observed MAE=0.029036 rank=54
[SoftImpute] Iter 2: observed MAE=0.029048 rank=54
[SoftImpute] Iter 3: observed MAE=0.029059 rank=54
[SoftImpute] Iter 4: observed MAE=0.029069 rank=54
[SoftImpute] Iter 5: observed MAE=0.029079 rank=54
[SoftImpute] Iter 6: observed MAE=0.029088 rank=54
[SoftImpute] Iter 7: observed MAE=0.029097 rank=54
[SoftImpute] Iter 8: observed MAE=0.029105 rank=54
[SoftImpute] Iter 9: observed MAE=0.029113 rank=54
[SoftImpute] Iter 10: observed MAE=0.029121 rank=54
[SoftImpute] Iter 11: observed MAE=0.029128 rank=54
[SoftImpute] Iter 12: observed MAE=0.029135 rank=54
[SoftImpute] Iter 13: observed MAE=0.029141 rank=54
[SoftImpute] Iter 14: observed MAE=0.029147 rank=54
[SoftImpute] Iter 15: observed MAE=0.029152 rank=54
[SoftImpute] Iter 16: observed MAE=0.029158 rank=54
[SoftImpute] Iter 17: observed MAE=0.029162 rank=54
[SoftImpute] Iter 18: observed MAE=0.029167 rank=54
[SoftImpute] Iter 1

array([[1.000e-03, 0.000e+00, 1.205e+00],
       [2.000e-03, 0.000e+00, 1.591e+00],
       [3.000e-03, 0.000e+00, 1.859e+00],
       [4.000e-03, 0.000e+00, 1.887e+00],
       [5.000e-03, 0.000e+00, 1.893e+00]])